In [6]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [7]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [8]:
print("API Key prefix:", os.getenv("OPENAI_API_KEY")[:20]) 

API Key prefix: 


In [9]:
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
resp = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello from test"}]
)

print(resp.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [11]:
# A class to represent a Webpage
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        print('The response is:\n ', response)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        #print('Soup body:/n ', soup.body)
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [21]:
mysite = Website("https://www.playstation.com/en-us")

The response is:
  <Response [200]>


In [22]:
print(mysite.title)

PlayStation® Official Site: Consoles, Games, Accessories & More


In [27]:
#print(mysite.text)

In [12]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [13]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [19]:
# print(user_prompt_for(mysite))

In [14]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [29]:
#messages_for(mysite)

In [15]:
# calling the OpenAI API (completions API):

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini", # will cost fractions of a cent!
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://www.playstation.com/en-us")

The response is:
  <Response [200]>


"# PlayStation® Official Site Summary\n\nThe PlayStation® Official Site showcases a variety of products and promotions for its gaming consoles, games, accessories, and subscription services, primarily focusing on the PlayStation 5 (PS5).\n\n## Key Highlights:\n\n### Upcoming Releases\n- **Call of Duty: Black Ops 7**: Set to be released on **November 14, 2025**, offering a co-op campaign, multiplayer, and Zombies mode.\n- **Where Winds Meet**: An epic open-world RPG launching on **November 14, 2025**.\n- **Mafia: The Old Country**: Currently available on PS5.\n- Other notable titles include **Death Stranding 2**, **Metal Gear Solid Δ**, and **Ghost of Yōtei Legends DLC**.\n\n### Promotions\n- Pre-order options are available for upcoming games and significant savings are currently offered on various titles through a PlayStation Store promotion.\n\n### Accessories\n- A comprehensive list of PS5 consoles and accessories is available, including:\n  - **DualSense™ Wireless Controller**\n  - 

In [17]:
# function to display the above returned content in a readable manner:

In [19]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
# considering the PlayStation's open website:
display_summary("https://www.playstation.com/en-us")

The response is:
  <Response [200]>


# PlayStation® Official Site Summary

The **PlayStation® Official Site** offers a range of information and promotions regarding its consoles, games, and accessories. Key highlights include:

## Upcoming Titles:
- **Call of Duty: Black Ops 7**: Launching on PS5 on **November 14**. Pre-orders are now available.
- **Where Winds Meet**: An epic Wuxia action-adventure RPG launching on **November 14, 2025**.
  
## Current Games & Promotions:
- A special promotion offers big savings on various titles in the PlayStation Store.
- Notable releases available now include **Madden NFL 26**, **Mafia: The Old Country**, and **Sword of the Sea**.

## PlayStation Plus:
- Discover a variety of plans including **Essential**, **Extra**, and **Premium**, offering access to hundreds of games, online multiplayer, and exclusive discounts.

## Accessory Offerings:
- Features of the PS5 console and accessories are highlighted, including the **DualSense™ Wireless Controller**, **PULSE Elite™ Wireless Headset**, and **Access™ Controller** aimed at inclusivity.

## 30-Year Celebration:
- PlayStation is marking its 30th anniversary with special events and a retrospective on its history.

## Recent News Highlights:
- **Price Increase Announcement**: The recommended retail price for PS5 consoles in the U.S. will increase effective **August 21, 2025**.
- **New Game Trailers**: Recent titles such as **Resident Evil Requiem** and **Onimusha: Way of the Sword** were showcased at Gamescom with upcoming releases expected in 2026.

This site serves as a comprehensive platform for PlayStation enthusiasts, detailing current offerings, news, and future releases in the gaming world.